## Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

[LangChain](https://python.langchain.com) - это набирающая популярность библиотека для работы с большими языковыми моделями и для построения конвейеров обработки текстовых данных. В одной библиотеке присутствуют все элементы, которые помогут нам создать вопрос-ответного бота на наборе текстовых данных: вычисление эмбеддингов, запуск больших языковых моделей для генерации текста, поиск по ключу в векторных базах данных и т.д.

Для начала, установим `langchain` и сопутствующие библиотеки.

In [16]:
%pip install -q langchain sentence_transformers lancedb unstructured

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account al

## Разбиваем документ на части

Для работы retrival augmented generation нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка LangChain - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

> **ВАЖНО**: Перед выполнением ячейки не забудьте установить имя пользователя, которое вы использовали на предыдущем шаге.

Размер `chunk_size` нужно выбирать исходя из нескольких показателей:
* Допустимая длина контекста для эмбеддинг-модели. Yandex GPT Embeddings допускают 2048 токенов, в то время как многие открытые модели HuggingFace имеют длину контекста 512-1024 токена
* Допустимый размер окна контекста большой языковой модели. Если мы хотим использовать в запросе top 3 результатов поиска, то 3 * chunk_size+prompt_size+response_size должно не превышать длины контекста модели.

In [19]:
import langchain
import langchain.document_loaders

user = 'shwars'
chunk_size = 1024
chunk_overlap=50
source_dir = f"/home/jupyter/datasphere/s3/s3store/{user}/text"

loader = langchain.document_loaders.DirectoryLoader(source_dir,glob="*.txt",show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
fragments = splitter.create_documents([ x.page_content for x in loader.load()])
len(fragments)

  0%|          | 0/3 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 3/3 [00:05<00:00,  1.98s/it]


131

## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов можно взять какую-нибудь модель из репозитория HuggingFace, с поддержкой русского языка. LangChain содержит свои встроенные средства работы с эмбеддингами, и поддерживает модели из HuggingFace: 

In [20]:
embeddings = langchain.embeddings.HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

2023-09-06 11:24:45.490989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


512

Также можно использовать более продвинутую модель [эмбеддингов от Yandex GPT](https://cloud.yandex.ru/docs/yandexgpt/api-ref/Embeddings/). Вот так можно вызывать её через API Yandex Cloud. Не забудьте при необходимости исправить параметр `folder_id` в соответствии с вашими данными.

In [21]:
import os
api_key = os.environ['api_key']
folder_id = "b1g6krtrd2vcbunvjpg6"

In [22]:
import requests

headers={ 
    "Authorization" : f"Api-key {api_key}",
    "x-folder-id" : folder_id
}
j = {
  "model" : "general:embedding",
  "embedding_type" : "EMBEDDING_TYPE_DOCUMENT",
  "text": "Hello, world!"
}

res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/embedding",json=j,headers=headers)
vec = res.json()['embedding']
len(vec)

256

Чтобы работать с этими эмбеддингами в LangChain, необходимо реализовать соответствующий адаптер. Это упрощенная версия, здесь мы не делаем проверки на какие-либо ошибки. Кроме того, поскольку на данный момент есть ограничение на скорость запросов (1 запрос в секунду), то в явном виде добавлена задержка между запросами.

In [23]:
from langchain.embeddings.base import Embeddings
import time

class YaGPTEmbeddings(Embeddings):

    def __init__(self,folder_id,api_key,sleep_interval=1):
        self.folder_id = folder_id
        self.api_key = api_key
        self.sleep_interval = sleep_interval
        self.headers = { 
                        "Authorization" : f"Api-key {api_key}",
                        "x-folder-id" : folder_id }
        
    def embed_document(self, text):
        j = {
          "model" : "general:embedding",
          "embedding_type" : "EMBEDDING_TYPE_DOCUMENT",
          "text": text
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/embedding",json=j,headers=self.headers)
        vec = res.json()['embedding']
        return vec

    def embed_documents(self, texts, chunk_size = 0):
        res = []
        for x in texts:
            res.append(self.embed_document(x))
            time.sleep(self.sleep_interval)
        return res
        
    def embed_query(self, text):
        j = {
          "model" : "general:embedding",
          "embedding_type" : "EMBEDDING_TYPE_QUERY",
          "text": text
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/embedding",json=j,headers=self.headers)
        vec = res.json()['embedding']
        return vec
    
embeddings = YaGPTEmbeddings(folder_id,api_key)
#res = embeddings.embed_document('Hello')
res = embeddings.embed_documents(['Hello','there'])
len(res),len(res[0])

(2, 256)

## Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных. Для крупных проектов имеет смысл использовать серьезные инструменты, типа [OpenSearch](https://opensearch.org/) (доступный [в виде сервиса в облаке Yandex Cloud](https://cloud.yandex.ru/services/managed-opensearch)), но для нашего примера мы используем небольшую векторную БД [LanceDB](https://lancedb.com/), хранящую индекс в директории на диске.

Первоначально создадим базу данных, добавив туда одну строчку:

In [24]:
from langchain.vectorstores import LanceDB
import lancedb
import os

db_dir = "store"

os.makedirs(db_dir,exist_ok=True)

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Далее проиндексируем все выделенные ранее фрагменты нашей документации, используя реализованный нами выше адаптер для YandexGPT-эмбеддингов. 

> В зависимости от объема текста, эта ячейка может выполняться достаточно длительное время - вспомним про задержку в 1 сек между запросами!

In [25]:
db = LanceDB.from_documents(fragments, embeddings, connection=table)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [26]:
q="Почему стоит работать в Яндексе?"

res = db.similarity_search(q)
for x in res:
    print('-'*40)
    print(x.page_content)

----------------------------------------
самое важное в тех людях которые работают вместе с тобой над проектами над продуктами мы внутри яндекс класс ну на самом деле прям самое важное то есть я думаю ну понятно что я минимальный порог входа если вы не знаете формулу боец и хотите заниматься математикой и мэлин то возможно вы не пройдете наши собеседования но это не так критично а в том плане что это там можно изучить и тому подобное а есть важные человеческие качества которые мы тоже проверяем и во многом мы это проверяем например за счет того что студенты к нам не приходят сразу работать они там идут на проектах потом работали На проекте потом поработать на стажировке мы про них много чего узнали что не проверишь Нам очень важно чтобы человек во 1 был неприменимым программиста Ой был отменен Наоборот твое место это люди невы Смысл программиста в том чтобы автоматизировать И есть наши водятся чтобы автоматизировать рутину убрать людей Если человек готов Непрерывно делать 1 и ту же раб

Ещё один полезный интерфейс для поиска текстов - это `Retriever`, убедимся, что он тоже работает:

In [27]:
retriever = db.as_retriever(
    search_kwargs={"k": 5}
)
res = retriever.get_relevant_documents(q)
for x in res:
    print(x.page_content)

самое важное в тех людях которые работают вместе с тобой над проектами над продуктами мы внутри яндекс класс ну на самом деле прям самое важное то есть я думаю ну понятно что я минимальный порог входа если вы не знаете формулу боец и хотите заниматься математикой и мэлин то возможно вы не пройдете наши собеседования но это не так критично а в том плане что это там можно изучить и тому подобное а есть важные человеческие качества которые мы тоже проверяем и во многом мы это проверяем например за счет того что студенты к нам не приходят сразу работать они там идут на проектах потом работали На проекте потом поработать на стажировке мы про них много чего узнали что не проверишь Нам очень важно чтобы человек во 1 был неприменимым программиста Ой был отменен Наоборот твое место это люди невы Смысл программиста в том чтобы автоматизировать И есть наши водятся чтобы автоматизировать рутину убрать людей Если человек готов Непрерывно делать 1 и ту же работу которую можно за вечер затянуть Это н

## Подключаемся к Yandex GPT

Фреймворк LangChain поддерживает интеграцию с различными большими языковыми моделями, но Yandex GPT в их число не входит. Поэтому, как и в случае с эмбеддингами, нам надо написать соответствующий адаптер, предоставляющий доступ к Yandex GPT в формате LangChain. Для подробностей вызова Yandex GPT можно обратиться к документации по [YandexGPT API](https://cloud.yandex.ru/docs/yandexgpt/)

In [28]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests

class YandexLLM(langchain.llms.base.LLM):
    api_key: str = None
    iam_token: str = None
    folder_id: str
    max_tokens : int = 1500
    temperature : float = 1
    instruction_text : str = None

    @property
    def _llm_type(self) -> str:
        return "yagpt"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        headers = { "x-folder-id" : self.folder_id }
        if self.iam_token:
            headers["Authorization"] = f"Bearer {self.iam_token}"
        if self.api_key:
            headers["Authorization"] = f"Api-key {self.api_key}"
        req = {
          "model": "general",
          "instruction_text": self.instruction_text,
          "request_text": prompt,
          "generation_options": {
            "max_tokens": self.max_tokens,
            "temperature": self.temperature
          }
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
          headers=headers, json=req).json()
        return res['result']['alternatives'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"max_tokens": self.max_tokens, "temperature" : self.temperature }

Теперь попросим модель ответить на наш вопрос от лица сотрудника Yandex Cloud:

In [29]:
instructions = """
Представь себе, что ты сотрудник Yandex Cloud. Твоя задача - вежливо и по мере своих сил отвечать на все вопросы собеседника."""

llm = YandexLLM(api_key=api_key, folder_id=folder_id,
                instruction_text = instructions)

In [30]:
llm(q)

'Конечно, я с радостью расскажу вам о том, почему работа в Яндекс Клауд является необходимым и привлекательным выбором. Вот несколько причин:\n\n1. Разнообразие и инновации. В отличие от других компаний, Яндекс постоянно расширяет свою продуктовую линейку, предлагая инновационные решения и сети для работы. Это дает возможность сотрудникам работать над множеством проектов и получить опыт в различных областях.\n2. Развитие карьеры. Яндекс Клауд предоставляет своим сотрудникам огромные возможности для карьерного роста. Компания обеспечивает обучение, предоставление доступа к ресурсам и накопленный экспертизе, что позволяет людям непрерывно расти и развиваться.\n3. Неограниченные возможности. Яндекс ценит своих сотрудников как людей, способных решать любые задачи, профессиональные и личные. Компания учитывает индивидуальные интересы и потребности работников, целевая их целеустремленность и подход к работе.\n4. Сообщество и современные решения. Креативная и инновационная команда сотрудников

В данном примере мы пока что никак не использовали наши текстовые документы.

## Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в наш текстовый корпус. Для этого используем механизм цепочек (*chains*). Основную функциональность реализует `StuffDocumentsChain`, которая делает следующее:

1. Берёт коллекцию документов `input_documents`
1. Каждый из них пропускает через `document_prompt`, и затем объединяет вместе.
1. Данный текст помещается в переменную `document_variable_name`, и передаётся большой языковой модели `llm`

В нашем случае `document_prompt` не будет модицицировать документ, а основной запрос к LLM будет содержать в себе инструкции для Yandex GPT. 

In [31]:
# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=res, query=q)

'Если говорить о работе в Яндексе, то в первую очередь стоит отметить, что наша компания активно развивается и ставит перед собой амбициозные цели. Мы всегда уделяем внимание новым технологиям и стремимся быть лидерами в своей отрасли. Кроме того, мы ценим наших сотрудников и делаем всё возможное, чтобы создать комфортные условия для работы. Если вы интересуетесь технологиями, хотите развиваться и достигать новых успехов, то работа в Яндексе может стать отличной возможностью для этого.'

Чтобы ещё более улучшить результат, мы можем использовать хитрый механизм перемешивания документов, таким образом, чтобы наиболее значимые документы были ближе к началу запроса. Также мы оформим все операции в одну функцию `answer`:

In [32]:
from langchain.document_transformers import LongContextReorder
reorderer = LongContextReorder()

def answer(query,reorder=True,print_results=False):
  results = retriever.get_relevant_documents(query)
  if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [33]:
answer("Почему хорошо работать в Yandex?",print_results=True)

самое важное в тех людях которые работают вместе с тобой над проектами над продуктами мы внутри яндекс класс ну на самом деле прям самое важное то есть я думаю ну понятно что я минимальный порог входа если вы не знаете формулу боец и хотите заниматься математикой и мэлин то возможно вы не пройдете наши собеседования но это не так критично а в том плане что это там можно изучить и тому подобное а есть важные человеческие качества которые мы тоже проверяем и во многом мы это проверяем например за счет того что студенты к нам не приходят сразу работать они там идут на проектах потом работали На проекте потом поработать на стажировке мы про них много чего узнали что не проверишь Нам очень важно чтобы человек во 1 был неприменимым программиста Ой был отменен Наоборот твое место это люди невы Смысл программиста в том чтобы автоматизировать И есть наши водятся чтобы автоматизировать рутину убрать людей Если человек готов Непрерывно делать 1 и ту же работу которую можно за вечер затянуть Это н

'В Yandex очень важно, чтобы каждый член команды был готов принимать решения и ответственно подходить к выполнению своей работы. В команде царят дружелюбие и поддержка, коллектив старается помогать друг другу и поддерживать инициативу. Каждый сотрудник может внести свой вклад в общее дело и внести идеи, новые идеи приветствуются. Здесь нет дресс-кода и закрытых дверей. В помещении работает несколько компьютеров, множество принтеров, есть современная лаборатория. Наша цель как IT компании - развитие и внедрение инновационных решений в IT бизнес. Для этого сотрудники Яндекса должны постоянно совершенствовать свои навыки.\nЕсли ты хочешь попробовать себя в качестве члена команды Яндекса, пожалуйста, расскажи о своих навыках и опыте.'

Можно сравнить результаты, выдаваемые Yandex GPT напрямую, с результатами нашей генерации на основе документов:

In [36]:
def compare(q):
    print(f"Ответ YaGPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")
    
compare("Что ты можешь сказать об ML-команде Яндекс-облака?")

Ответ YaGPT: С удовольствием отвечу на ваши вопрос по ML команде Яндекс облака.
Ответ бота: Конечно! Что вы хотите узнать о нашей ML-команде?


## Сохраняем векторную БД в Storage

Для следующего этапа - вопрос-ответного бота - нам потребуется созданная нами база данных с документами. Поэтому скопируем её на хранилище s3:

In [37]:
!cp -R ./store /home/jupyter/datasphere/s3/s3store/shwars/